In [1]:
import numpy as np
from sklearn.pipeline import Pipeline

import GEOparse
from Genobolitics import *
from sklearn_utils.preprocessing import FoldChangeScaler
from metabolitics.preprocessing import MetaboliticsTransformer, MetaboliticsPipeline

import pickle

In [2]:
import sys, cplex

sys.path

['',
 '/home/abdu/Desktop/virtualenv/GERA/lib/python35.zip',
 '/home/abdu/Desktop/virtualenv/GERA/lib/python3.5',
 '/home/abdu/Desktop/virtualenv/GERA/lib/python3.5/plat-x86_64-linux-gnu',
 '/home/abdu/Desktop/virtualenv/GERA/lib/python3.5/lib-dynload',
 '/usr/lib/python3.5',
 '/usr/lib/python3.5/plat-x86_64-linux-gnu',
 '/home/abdu/Desktop/virtualenv/GERA/lib/python3.5/site-packages',
 '/home/abdu/Desktop/virtualenv/GERA/lib/python3.5/site-packages/IPython/extensions',
 '/home/abdu/.ipython']

In [3]:
breast_cancer_dataset = GEOparse.get_GEO('GDS5819').table

06-Jan-2018 15:54:10 INFO GEOparse - File already exist: using local version.
06-Jan-2018 15:54:10 INFO GEOparse - Parsing ./GDS5819.soft.gz: 
06-Jan-2018 15:54:10 DEBUG GEOparse - DATABASE: Geo
06-Jan-2018 15:54:10 DEBUG GEOparse - DATASET: GDS5819
06-Jan-2018 15:54:10 DEBUG GEOparse - SUBSET: GDS5819_1
06-Jan-2018 15:54:10 DEBUG GEOparse - SUBSET: GDS5819_2
06-Jan-2018 15:54:10 DEBUG GEOparse - SUBSET: GDS5819_3
06-Jan-2018 15:54:10 DEBUG GEOparse - SUBSET: GDS5819_4
06-Jan-2018 15:54:10 DEBUG GEOparse - DATASET: GDS5819
/home/abdu/Desktop/virtualenv/GERA/lib/python3.5/site-packages/GEOparse/GEOparse.py:330 FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls


In [4]:
unhealthy_raw='\
#GSM1599177 = Value for GSM1599177: Primary human monocytes from patient with metastatic breast cancer #1; src: Primary human monocytes\
#GSM1599178 = Value for GSM1599178: Primary human monocytes from patient with metastatic breast cancer #2; src: Primary human monocytes\
#GSM1599179 = Value for GSM1599179: Primary human monocytes from patient with metastatic breast cancer #3; src: Primary human monocytes\
#GSM1599180 = Value for GSM1599180: Primary human monocytes from patient with metastatic breast cancer #4; src: Primary human monocytes'

In [6]:
healthy_raw='\
#GSM1599181 = Value for GSM1599181: Primary human monocytes from healthy control #1; src: Primary human monocytes\
#GSM1599182 = Value for GSM1599182: Primary human monocytes from healthy control #2; src: Primary human monocytes\
#GSM1599183 = Value for GSM1599183: Primary human monocytes from healthy control #3; src: Primary human monocytes'

In [7]:
clean = lambda string: string.replace(':', '').split()
predicate = lambda word: word.startswith('GSM')

healthy = set(filter(predicate, clean(healthy_raw)))
unhealthy = set(filter(predicate, clean(unhealthy_raw)))

In [8]:
print(len(healthy), len(unhealthy))

3 4


In [9]:
healthy & unhealthy

set()

In [10]:
breast_cancer_dataset[breast_cancer_dataset.columns[:5]].head()

,ID_REF,IDENTIFIER,GSM1599177,GSM1599178,GSM1599179
0,ILMN_1343048,ILMN_1343048,NaN,NaN,NaN
1,ILMN_1343049,ILMN_1343049,NaN,NaN,NaN
2,ILMN_1343050,ILMN_1343050,NaN,NaN,NaN
3,ILMN_1343052,ILMN_1343052,NaN,NaN,NaN
4,ILMN_1343059,ILMN_1343059,NaN,NaN,NaN


In [11]:
labels_ = [(h, 'healthy') for h in healthy] + [(d, 'unhealthy') for d in unhealthy]

In [ ]:
X, y = parse_database('GDS5819', labels=dict(labels_))

In [13]:
pickle.dump(X, open("datasets/breast_cancer_X","wb"))
pickle.dump(y, open("datasets/breast_cancer_y","wb"))

In [3]:
X = pickle.load(open("datasets/breast_cancer_X","rb"))
y = pickle.load(open("datasets/breast_cancer_y","rb"))

In [4]:
genobolitics_transformer = MetaboliticsTransformer()
genobolitics_transformer.analyzer = Genobolitics("recon2")

pipeline = Pipeline([('scaling', FoldChangeScaler(reference_label='healthy')),
                     ('LP_FVA', genobolitics_transformer)])

results = pipeline.fit_transform(X, y)

pickle.dump(results, open("results/breast_cancer.results", "wb"))

/home/abdu/Desktop/GERA/Genobolitics.py:35 UserWarning: some operands are missing from logical expression!
/home/abdu/Desktop/GERA/Genobolitics.py:53 UserWarning: could not evaluate boolean expression, objective-coeff is set to ZERO!
/home/abdu/Desktop/GERA/Genobolitics.py:69 UserWarning: model returned the empty string as reaction rule!
/home/abdu/Desktop/GERA/Genobolitics.py:35 UserWarning: some operands are missing from logical expression!
/home/abdu/Desktop/GERA/Genobolitics.py:53 UserWarning: could not evaluate boolean expression, objective-coeff is set to ZERO!
/home/abdu/Desktop/GERA/Genobolitics.py:69 UserWarning: model returned the empty string as reaction rule!
/home/abdu/Desktop/GERA/Genobolitics.py:35 UserWarning: some operands are missing from logical expression!
/home/abdu/Desktop/GERA/Genobolitics.py:53 UserWarning: could not evaluate boolean expression, objective-coeff is set to ZERO!
/home/abdu/Desktop/GERA/Genobolitics.py:69 UserWarning: model returned the empty strin

In [7]:
results

[{'ENMAN6g_max': 1.1368683772161603e-11,
  'RE1916C_min': 0.0,
  'EX_psylchol_LPAREN_e_RPAREN__max': 2.2680524125462398e-10,
  'ALADGLNexR_max': -999.9999999998113,
  'PCHOL_HStg_max': 1000.0,
  'LCYSTAT_max': -1.1823431123048067e-11,
  'r1781_min': 0.0,
  'r1687_max': 4.695266397902742e-11,
  'EX_maltttr_LPAREN_e_RPAREN__min': -491.8927106643746,
  'C4CRNCPT2_max': 468.01593959784054,
  'EX_oh1_LPAREN_e_RPAREN__max': 0.0,
  'RE3020C_min': 0.0,
  'RE3526M_max': 0.0,
  'G14T17g_min': 0.0,
  'PHETHPTOX2_max': 1.6789272194728255e-09,
  'RETH2e_max': -2.708588908717502e-11,
  'NDPK10_min': 8.947995345129307e-10,
  'PTE3x_min': 0.0,
  'BAMPPALDOXm_min': 0.0,
  'EX_dspvs_LPAREN_e_RPAREN__min': 0.0,
  'PGDIr_max': 0.0,
  'r1076_max': 0.0,
  'PTDCAt_min': -2.892193151637912e-10,
  'EX_3hpvstet_LPAREN_e_RPAREN__max': 1.4949819160392508e-11,
  'NDPK10n_min': -1000.0,
  'DOPAQNISO1_min': 0.0,
  'RE3041N_max': 0.0,
  'P45021A2r_max': 7.673861546209082e-13,
  'EX_3hsmvacid_LPAREN_e_RPAREN__max': 3.